# PyTorch の学習と推論を Amazon SageMaker で行う

MNISTデータセットを対象にAmazon SageMakerとPyTorchを利用してCNNの学習と推論を行います。学習の方法として以下の3種類を試します。
- 単一または複数ノードによる学習: 高性能なトレーニングインスタンスを1つまたは複数立ち上げて学習を行います。
- ハイパーパラメータ最適化: 単一ノードにおける学習でHPO (Hyper-Parameter Optimization: ハイパーパラメータ最適化）を行います。
- ローカルモード: ノートブックインスタンスで学習します。追加のインスタンス立ち上げが不要で、開発時のデバッグに有用です。


## 目次
1. [準備](#準備)
2. [データの取得とS3へのアップロード](#データの取得とS3へのアップロード)
3. [エントリーポイント](#エントリーポイント)
4. [モデルの学習](#モデルの学習)
    1. [単一または複数ノードによる学習](#単一または複数ノードによる学習)
    2. [ハイパーパラメータの最適化](#ハイパーパラメータの最適化)
    3. [ローカルモード](#ローカルモード)
5. [学習結果の可視化](#学習結果の可視化)
6. [ハイパーパラメータのチューニング結果](#ハイパーパラメータのチューニング結果)
7. [モデルの推論を実行](#モデルの推論を実行)
8. [エンドポイントの削除](#エンドポイントの削除)

## 準備

ローカルモードを実行するため、いくつかのパッケージを事前インストールする必要があります。そのためのスクリプト`setup.sh`を用意しているので実行しましょう。

In [ ]:
!sh setup.sh

## データの取得とS3へのアップロード

ここでは、`PyTorch` でサポートされている関数を使って、MNIST データをダウンロードします。SageMaker の学習時に利用するデータは、S3 に置く必要があります。ここでは、ローカルにダウンロードした MNIST データを npz 形式で固めてから、SageMaker のラッパー関数を使って S3 にアップロードします。

デフォルトでは SageMaker は `sagemaker-{region}-{your aws account number}` というバケットを使用します。当該バケットがない場合には、自動で新しく作成します。`upload_data()` メソッドの引数に bucket=XXXX という形でデータを配置するバケットを指定することも可能です。

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
from torchvision import datasets, transforms

datasets.MNIST('data', download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))


prefix = 'sagemaker/DEMO-pytorch-mnist'
inputs = sagemaker_session.upload_data(path='data', key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

## エントリーポイント

SageMakerで、Pytroch、Chainer、Tensorflowなどのフレームワークを利用して深層学習を行うためには、このnotebook以外に**エントリーポイントを作成する必要があります**。エントリーポイントとはモデルや学習方法を記述した.pyファイルで、このnotebookには`mnist.py`というエントリーポイントを同じフォルダに用意しています。ノートブックインスタンスでfit関数を呼び出すと、エントリーポイントに沿って学習が行われます。

PyTorchを利用する場合は、エントリーポイントの`__main__`関数内にモデルの記述や学習方法を記載すればよく、SageMakerを使う以前のPyTorchのコードを概ねそのまま利用することができます。また、環境変数経由で入力データの場所や GPU の数などを取得することが可能です。これは `argparse` 経由で `main` 関数内で受け取ることができます。詳細は[こちら](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/pytorch/README.rst)をご覧ください。

また推論時の処理は、`model_fn` で学習済みモデルをロードする部分だけ記述する必要があります。その他オプションで、前処理、推論処理、後処理部分を `input_fn`、 `predict_fn`、 `output_fn` で書くこともできます。デフォルトでは、`application/x-npy` Content-Typeで指定される、NumPy 配列を入力として受け取ります。 

以下のセルを実行してエントリーポイントの中身を表示してみます。すると、`class Net(nn.Module)`内でのCNNの定義、`__main__`の中に学習のコードが書かれていることがわかります。

In [ ]:
!pygmentize 'mnist.py'

## モデルの学習

`Estimator` クラスの子クラスの `PyTorch` オブジェクトを作成し、`fit()` メソッドで学習ジョブを実行します。 `entry_point` で指定したローカルのスクリプトが、学習用のコンテナ内で実行されます。また合わせてローカルの `source_dir` を指定することで、依存するスクリプト群をコンテナにコピーして、学習時に使用することが可能です。`source_dir`にrequirements.txtという名前で、pipでインストール可能なライブラリのリストを入れておくと、学習時にpipで自動インストールされます。

### 単一または複数ノードによる学習

単一ノードで学習したい場合は、`train_instance_count=1`として、学習用インスタンスを`instance_type`に指定します。複数ノードによる学習は、`train_instance_count`を1より大きくすることで実行できます。複数ノードの場合には、エントリーポイントに分散学習となるような実装が必要になります。PyTorchではCPUの分散学習のためにgloo、GPUの分散学習のためにncclを選ぶことができます。あとで学習の結果を参照するためにジョブの名前を記録しておきます。

In [ ]:
from sagemaker.pytorch import PyTorch

instance_type = 'ml.c4.xlarge'
pytorch_estimator = PyTorch(entry_point='mnist.py',
                    source_dir =".",
                    role=sagemaker.get_execution_role(),
                    framework_version='0.4.0',
                    train_instance_count=2,
                    train_instance_type=instance_type,
                    hyperparameters={
                        'epochs': 6,
                        'backend': 'gloo'
                    })

pytorch_estimator.fit({'training': inputs})

# Keep the job name for checking training loss later 
training_job = pytorch_estimator.latest_training_job.name

### ハイパーパラメータの最適化

ハイパーパラメータの最適化は、fitする前に、以下のような処理を書くことによって実行できます。

- ハイパーパラメータの探索条件の設定
    - カテゴリ変数、連続変数、離散変数かどうか、探索範囲の指定を行います。
- ハイパーパラメータを選択する基準（以下では、バリデーションデータに対する精度で選択）
    - エントリーポイント内のPrintReportでバリデーションデータに対する精度を出力 
    - ログからバリデーションデータに対する精度のみを抽出する正規表現を記述  
    
- 上記の探索範囲、選択基準、チューニングのために実行するジョブ数などを指定してHyperparameterTunerを定義

ハンズオンの時間の都合上、以下では最適化のアルゴリズムをSGDかAdamのどちらかを選択するだけのチューニングを行います。
チューニングのジョブを以下のページで確認することができます。`Completed`になるまで5分程度かかりますので、待たずに次のローカルモードに進みましょう。  
https://ap-northeast-1.console.aws.amazon.com/sagemaker/home?region=ap-northeast-1#/hyper-tuning-jobs

In [ ]:
import subprocess
from sagemaker.pytorch import PyTorch
instance_type = 'ml.m4.xlarge'
pytorch_estimator = PyTorch(entry_point='mnist.py',
                    role=sagemaker.get_execution_role(),
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters={
                        'epochs': 6,
                        'backend': 'gloo'
                    })

###Setting for hyper paramter optimization###
from sagemaker.tuner import HyperparameterTuner,  CategoricalParameter, ContinuousParameter, IntegerParameter

hyperparameter_ranges = {'optimizer': CategoricalParameter(['sgd', 'Adam'])}
'''
An example of further tuning:
hyperparameter_ranges = {'optimizer': CategoricalParameter(['sgd', 'Adam']),
                          'learning_rate': ContinuousParameter(0.01, 0.2),
                          'num_epoch': IntegerParameter(3, 5)}
'''

objective_metric_name = 'Validation-accuracy'
metric_definitions = [{'Name': 'Validation-accuracy',
                       'Regex': 'Accuracy: ([0-9\\.]+)'}]

tuner = HyperparameterTuner(pytorch_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=2,
                            max_parallel_jobs=2)
##################################

tuner.fit({'training': inputs})

training_job_tuning = tuner.latest_tuning_job.name

### モデルの自動選択

SageMakerが指定できるハイパーパラメータは、いわゆるDeep Learningのハイパーパラメータ以外にも、様々なものを探索することができます。ここでは、複数のモデルを定義して、そのモデルの中から最善のものを選ぶために、ハイパーパラメータ最適化を利用してみたいと思います。`mnist.py`では畳み込み層2層のNetと畳み込み層3層のNet2を定義しています。ここで、`CategoricalParameter(['Net', 'Net2'])`と指定して、どちらかを選ぶようにします。指定したモデルが選択されるように`minist.py`の中には以下のようなコードを書いておき、あとは先ほどと同じ要領でHyperparameterTunerに対するfitを行います。

```python
if args.model == "Net2":
    model = Net2().to(device)
else:
    model = Net().to(device)
```


In [ ]:
###Setting for hyper paramter optimization###
from sagemaker.tuner import HyperparameterTuner,  CategoricalParameter, ContinuousParameter, IntegerParameter

instance_type = 'ml.m4.xlarge'
pytorch_estimator = PyTorch(entry_point='mnist.py',
                    role=sagemaker.get_execution_role(),
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters={
                        'epochs': 6,
                        'backend': 'gloo'
                    })

hyperparameter_ranges = {'model': CategoricalParameter(['Net', 'Net2'])}

objective_metric_name = 'Validation-accuracy'
metric_definitions = [{'Name': 'Validation-accuracy',
                       'Regex': 'Accuracy: ([0-9\\.]+)'}]

model_selector = HyperparameterTuner(pytorch_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=2,
                            max_parallel_jobs=2,
                            base_tuning_job_name ='model-selector-mnist')
##################################

model_selector.fit({'training': inputs})


### ローカルモード

ノートブックインスタンスのCPUで学習する場合は`instance_type = 'local'`、GPUで学習する場合は`local_gpu`を指定します。インスタンス数は、ノートブックインスタンスの数、すなわち1になるため、 `train_instance_count`に指定された値が1より大きい場合も1として扱われます。

In [ ]:
import subprocess

from sagemaker.pytorch import PyTorch

instance_type = 'local'

pytorch_estimator = PyTorch(entry_point='mnist.py',
                    source_dir =".",
                    role=sagemaker.get_execution_role(),
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters={
                        'epochs': 6,
                        'backend': 'gloo'
                    })


pytorch_estimator.fit({'training': inputs})

## ハイパーパラメータ最適化結果の確認

学習が終わったら結果を可視化してみましょう。その前に以下でチューニングのジョブが`complete`になっていることを確認します。チューニングのジョブの表で、先ほど実行したジョブの名前をクリックすると、トレーニングジョブのページに移動します。SGDとAdamのそれぞれの最適化を実行したジョブの結果が表示されており、それぞれの検証スコア（バリデーションデータに対する精度）が表示されていると思います。  
https://ap-northeast-1.console.aws.amazon.com/sagemaker/home?region=ap-northeast-1#/hyper-tuning-jobs  

`describe_training_job`を利用することで、ジョブの詳細を辞書形式で見ることができます。特に、チューニングジョブの詳細を見ることによって、ハイパーパラメータのチューニング結果を知ることができます。例えば、ハイパーパラメータ最適化によって、選択されたOptimizerを知りたい場合は、辞書の\['HyperParameters'\]\['Optimizer'\]を見ます。

In [ ]:
desc = tuner.sagemaker_session.sagemaker_client. \
           describe_training_job(TrainingJobName=tuner.best_training_job())
selected_optimizer = desc['HyperParameters']['optimizer']
print(selected_optimizer)

# モデルの推論を実行


推論を行うために学習したモデルをデプロイします。ここでは、ハイパーパラメータをチューニングした結果からデプロイしましょう。`deploy()` メソッドでは、デプロイ先エンドポイントのインスタンス数、インスタンスタイプを指定します。こちらもインスタンスタイプを `local` にすることで，このインスタンス内にエンドポイントを作成します。

In [ ]:
instance_type = 'ml.m4.xlarge'
predictor = tuner.deploy(initial_instance_count=1, instance_type=instance_type)

デプロイが終わったら実際に手書き文字認識を行ってみましょう。最初はランダムに5枚選んで推論をしてみます。

In [ ]:
%matplotlib inline
import random

import matplotlib.pyplot as plt

test_dataset = datasets.MNIST('data', train=False, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))


num_samples = 5
indices = random.sample(range(len(test_dataset) - 1), num_samples)
images = []
labels = []
for i in list(indices):
    image, label = test_dataset[i] 
    images.append(image.data.numpy())
    labels.append(label.data.numpy())
    
for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')

prediction = predictor.predict(images)
predicted_label = prediction.argmax(axis=1)
print('The predicted labels are: {}'.format(predicted_label))

次のセルを実行すると、HTMLのcanvasを表示して、枠内に手書きの数字を書くことができます。さらに次のセルを実行すると、キャンバスに書かれた数字に対して、エンドポイントで予測が実行されます。

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

In [ ]:
import numpy as np
image = np.array(data, dtype=np.float32)
prediction = predictor.predict([image])
predicted_label = prediction.argmax(axis=1)[0]
print('What you wrote is: {}'.format(predicted_label))

## エンドポイントの削除

不要になったエンドポイントを削除します．

In [ ]:
tuner.delete_endpoint()